In [157]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
warnings.filterwarnings(action='ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'

from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
from tqdm.notebook import tqdm

import datetime
from datetime import timedelta

import requests
from bs4 import BeautifulSoup

from datetime import datetime
import re

----------------------------

### 네이버 기사 제목, 링크, 날짜, 본문 내용 크롤링 코드

In [200]:
title_text=[]
link_text=[]
source_text=[]
date_text=[]
contents_text=[]

RESULT_PATH ='C:/Users/SEC/Desktop/Covid'  #결과 저장할 경로
now = datetime.now() #파일이름 현 시간으로 저장하기

#날짜 정제화 함수
def date_cleansing(test):
    try:
        #지난 뉴스
        #머니투데이  10면1단  2018.11.05.  네이버뉴스   보내기  
        pattern = '\d+.(\d+).(\d+).'  #정규표현식 
    
        r = re.compile(pattern)
        match = r.search(test).group(0)  # 2018.11.05.
        date_text.append(match)
        
    except AttributeError:
        #최근 뉴스
        #이데일리  1시간 전  네이버뉴스   보내기  
        pattern = '\w* (\d\w*)'     #정규표현식 
        
        r = re.compile(pattern)
        match = r.search(test).group(1)
        #print(match)
        date_text.append(match)


#내용 정제화 함수 
def contents_cleansing(contents):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '', 
                                      str(contents)).strip()  #앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '', 
                                       first_cleansing_contents).strip()#뒤에 필요없는 부분 제거 (새끼 기사)
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    contents_text.append(third_cleansing_contents)
    #print(contents_text)

def crawler(maxpage,query,sort,s_date,e_date):

    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1  
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지
    
    while page <= maxpage_t:
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        
        response = requests.get(url,headers={'User-Agent':'Mozilla/5.0'})
        html = response.text
 
        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')
 
        #<a>태그에서 제목과 링크주소 추출
        atags = soup.select('._sp_each_title')
        for atag in atags:
            title_text.append(atag.text)     #제목
            link_text.append(atag['href'])   #링크주소
            
        #신문사 추출
        source_lists = soup.select('._sp_each_source')
        for source_list in source_lists:
            source_text.append(source_list.text)    #신문사
        
        #날짜 추출 
        date_lists = soup.select('.txt_inline')
        for date_list in date_lists:
            test=date_list.text   
            date_cleansing(test)  #날짜 정제 함수사용 
        
        #본문요약본
        contents_lists = soup.select('ul.type01 dl')
        for contents_list in contents_lists:
            #print('==='*40)
            #print(contents_list)
            contents_cleansing(contents_list) #본문요약 정제화
        
        
        #모든 리스트 딕셔너리형태로 저장
        result= {"date" : date_text , "title":title_text ,  "source" : source_text ,"contents": contents_text ,"link":link_text }  
        print(page)
        
        df = pd.DataFrame(result)  #df로 변환
        page += 10
    

def main():
    info_main = input("="*50+"\n"+"입력 형식에 맞게 입력해주세요."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    
    maxpage = input("최대 크롤링할 페이지 수 입력하시오: ")  
    query = input("검색어 입력: ")  
    sort = input("뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): ")    #관련도순=0  최신순=1  오래된순=2
    s_date = input("시작날짜 입력(2019.01.04):")  #2019.01.04
    e_date = input("끝날짜 입력(2019.01.05):")   #2019.01.05
    
    crawler(maxpage,query,sort,s_date,e_date) 
    
main()

입력 형식에 맞게 입력해주세요.
 시작하시려면 Enter를 눌러주세요.
최대 크롤링할 페이지 수 입력하시오: 10
검색어 입력: 코로나
뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): 0
시작날짜 입력(2019.01.04):2020.02.01
끝날짜 입력(2019.01.05):2020.02.02
1
11
21
31
41
51
61
71
81
91


## 빅 카인즈 크롤링

In [2]:
contents = [] # 본문내용
title = [] # 주제
date = [] # 날짜
news = [] # 언론사
section = [] # 부문

In [ ]:
# 빅 카인즈 페이지 접속
driver = webdriver.Chrome("chromedriver.exe")
driver.get("https://www.bigkinds.or.kr/")

topic = "코로나"

# 주제어 입력
elem = driver.find_element_by_id("total-search-key")
elem.send_keys(topic)

# 검색 버튼 클릭
elem = driver.find_element_by_xpath("//*[@id='news-search-form']/div/div/div/div[1]/span/button")
elem.click()
time.sleep(3.5)

In [ ]:
# 페이지 순환하기
#for i in range(7,10):
elem = driver.find_element_by_xpath("//*[@id='news-results-pagination']/ul/li[9]/a")
elem.click()
time.sleep(3.5)
    # 페이지 내 10개 기사 크롤링
for i in range(1,101):

    # 신문사 크롤링
    news_elem = driver.find_element_by_xpath("//*[@id='news-results']/div["+str(i)+"]/div[2]/div[2]")
    news.append(news_elem.text[:4])
    time.sleep(0.1)

    # 섹션 크롤링
    section_elem = driver.find_element_by_xpath("//*[@id='news-results']/div["+str(i)+"]/div[2]/div[2]/span[1]")
    section.append(section_elem.text)
    time.sleep(0.1)

    # 날짜 크롤링 
    date_elem = driver.find_element_by_xpath("//*[@id='news-results']/div[" + str(i) +"]/div[2]/div[2]/span[2]")
    #date_elem = driver.find_element_by_class_name("news-item__date")
    date.append(date_elem.text)
    time.sleep(0.2)

    # 제목 크롤링
    title_elem = driver.find_element_by_xpath("//*[@id='news-results']/div[" +str(i) +"]/div[2]/h4")
    title.append(title_elem.text)
    time.sleep(0.2)

    # 기사 클릭
    elem = driver.find_element_by_xpath("//*[@id='news-results']/div[" + str(i) +"]/div[2]")
    elem.click()
    time.sleep(2.5)

    # 본문 크롤링
    elem = driver.find_element_by_xpath("//*[@id='news-detail-modal']/div/div/div[2]/div")
    content_elem = driver.find_element_by_class_name("news-detail__content")
    contents.append(content_elem.text)
    time.sleep(1.5)

    # 기사 END 버튼 클릭
    elem = driver.find_element_by_xpath("//*[@id='news-detail-modal']/div/div/div[1]/button")
    elem.click()
    time.sleep(1.5)
    print(i)

time.sleep(0.5)

# 35페이지 코로나충격최소화

In [191]:
len(title)
len(date)
len(contents)
len(news)
len(section)

57

57

57

57

57

In [180]:
df = pd.DataFrame({"제목" : title,
                   "날짜" : date,
                   "부문" : section,
                   "신문사" : news,
                   "본문" :contents})

In [21]:
df.to_excel('data.xlsx')

In [55]:
df2 = pd.DataFrame({"제목" : title,
                   "날짜" : date,
                   "부문" : section,
                   "신문사" : news,
                   "본문" :contents})

In [201]:
df2

,날짜,본문,부문,신문사,제목
0,2020/02/01,"[아무튼, 주말- saturday's pick]\n넷플릭스 | 판데믹\n\n""이런 ...",문화>전시_공연 | 문화>영화 | 문화>출판,조선일보,끔찍하면서도 훌륭한 타이밍…전염병과의 전쟁 외
1,2020/02/01,"文 부정평가 50%, 조국사태때 수준… 우한 폐렴, 불경기, 주택정책 영향 여성·2...",정치>청와대 | 정치>선거 | 정치>국회_정당,조선일보,"與 지지율 34%로 급락, 화들짝 놀란 당청"
2,2020/02/01,“한국인도 과거에 박쥐 먹었다” “설현도 박쥐를 먹었다”\n\n신종 코로나바이러스 ...,국제>일본 | 국제>중국 | 국제>아시아,국민일보,황교익 “혐오 말자는데 설현 얘기만…내 정치 성향 탓에 겪는 일”
3,2020/02/01,[커버스토리] 고대부터 인류 괴롭혀 온 전염병의 역사\n인류는 고대부터 무시무시한 ...,국제>유럽_EU | 국제>아시아 | 국제>중남미,국민일보,사상 최악 ‘스페인 독감’에 1억명 희생… 오늘도 진화하는 바이러스
4,2020/02/01,"진천·아산 주민들, 수용 반대 철회… 응원 현수막·SNS 캠페인\n“우리국민 어찌 ...",지역>지역일반 | 사회>사회일반,한국일보,배척만 하기엔… 시민들 ‘우한 교민 포용’ 빛났다
...,...,...,...,...,...
3496,2020/03/31,일본 수도 도쿄(東京)에서 31일 코로나19 확진자가 78명 발생했다고 NHK가 보...,국제>일본 | 국제>아시아 | 국제>중국,경향신문,"""일본, 코로나 하루 감염자 200명 첫 돌파""...도쿄도 78명 최고치"
3497,2020/03/31,"경산 2인조 절도범, 어린이집 원장 차량 털어 마스크 57장 훔치는 등 절도행각, ...",사회>사건_사고 | 지역>경남 | 지역>강원,조선일보,[단독]도둑맞은 꼬마들의 마스크가 돌아왔어요
3498,2020/03/31,독일이 신종 코로나바이러스 감염증(코로나19) 확산을 막기 위해 공공장소에서 2명을...,국제>미국_북미 | 국제>유럽_EU | 국제>아시아,세계일보,유럽·美 ‘코로나 불길’ 활활… 美 사망자 3000명 넘어서
3499,2020/03/31,코로나19 영향으로 국내 업종 10곳 중 9곳이 실적 악화를 체감하고 있다는 조사 ...,경제>취업_창업 | 경제>증권_증시 | 경제>무역,세계일보,코로나 탓에… 업종 10개 중 9개 실적 악화 실감


In [182]:
df2 = pd.concat([df2,df]).reset_index().drop(["index"],axis=1)

In [ ]:
for i in range(1,10):
        
        # 신문사 크롤링
        news_elem = driver.find_element_by_class_name("//*[@id='news-results']/div["+str(i)+"]/div[2]/div[2]/a")
        news.append(news_elem.text)
        time.sleep(0.1)
        
        # 섹션 크롤링
        section_elem = driver.find_element_by_xpath("//*[@id='news-results']/div["+str(i)+"]/div[2]/div[2]/span[1]")
        section.append(section_elem.text)
        time.sleep(0.1)

        # 날짜 크롤링 
        date_elem = driver.find_element_by_xpath("//*[@id='news-results']/div[" + str(i) +"]/div[2]/div[2]/span[2]")
        #date_elem = driver.find_element_by_class_name("news-item__date")
        date.append(date_elem.text)
        time.sleep(0.2)
 
        # 제목 크롤링
        title_elem = driver.find_element_by_xpath("//*[@id='news-results']/div[" +str(i) +"]/div[2]/h4")
        title.append(title_elem.text)
        time.sleep(0.2)
        
        # 기사 클릭
        elem = driver.find_element_by_xpath("//*[@id='news-results']/div[" + str(i) +"]/div[2]")
        elem.click()
        time.sleep(2)

        # 본문 크롤링
        elem = driver.find_element_by_xpath("//*[@id='news-detail-modal']/div/div/div[2]/div")
        content_elem = driver.find_element_by_class_name("news-detail__content")
        contents.append(content_elem.text)
        time.sleep(0.5)

        # 기사 END 버튼 클릭
        elem = driver.find_element_by_xpath("//*[@id='news-detail-modal']/div/div/div[1]/button")
        elem.click()
        time.sleep(1.5)
        print(i)